<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >

# 理解 Python 的数据类型
# Understanding Data Types in Python

想要高效的处理数据驱动的科学计算就必须知道数据是如何被存储和操纵的。本章节阐述了 Python 是如何处理数据数组的，并介绍了 NumPy 如何对其进行了优化。

Effective data-driven science and computation requires understanding how data is stored and manipulated.
This section outlines and contrasts how arrays of data are handled in the Python language itself, and how NumPy improves on this.
Understanding this difference is fundamental to understanding much of the material throughout the rest of the book.

Python 用户经常被其易用所吸引，动态类型就是其易用性的一个体现。C、Java 这样的静态语言要求明确的声明每个变量的类型，而 Python 这样的动态类型语言则不必如此。在 C 语言中你可能需要写成下面这个样子：

Users of Python are often drawn-in by its ease of use, one piece of which is dynamic typing.
While a statically-typed language like C or Java requires each variable to be explicitly declared, a dynamically-typed language like Python skips this specification. For example, in C you might specify a particular operation as follows:

```C
/* C code */
int result = 0;
for(int i=0; i<100; i++){
    result += i;
}
```

而在 Python 的等价的运算你可以这样写：

While in Python the equivalent operation could be written this way:

```python
# Python code
result = 0
for i in range(100):
    result += i
```

可以看到主要的区别如下：

在 C 中，需要声明变量的数据类型，而 Python 中类型是被动态推断的。这意味着我们可以个任意一个变量赋予任意类型的数据：

Notice the main difference: in C, the data types of each variable are explicitly declared, while in Python the types are dynamically inferred. This means, for example, that we can assign any kind of data to any variable:

```python
# Python code
x = 4
x = "four"
```

这里我们把 ``x`` 的类型从整型变成了字符串。类似的行为在 C 语言中会导致一个编译错误或者其他意想不到的后果（取决于编译器的设定）：

Here we've switched the contents of ``x`` from an integer to a string. The same thing in C would lead (depending on compiler settings) to a compilation error or other unintented consequences:

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

类似的灵活性增加了 Python 这样的动态语言的易用性。理解它的运作机制是有效并高效的用 Python 做数据分析的重要内容。不过这种灵活类型也说明 Python 的变量不仅仅包含了数值，也包含了有关其类型的其他信息，在本章节我们将做进一步的阐述。

This sort of flexibility is one piece that makes Python and other dynamically-typed languages convenient and easy to use.
Understanding *how* this works is an important piece of learning to analyze data efficiently and effectively with Python.
But what this type-flexibility also points to is the fact that Python variables are more than just their value; they also contain extra information about the type of the value. We'll explore this more in the sections that follow.

## Python 整型不仅仅是整型

## A Python Integer Is More Than Just an Integer

Python 语言本身由 C 语言编写，这意味着每一个 Python 的变量就是一个 C 的结构体，因此它必定包含着除数值外的其他信息。例如当我们在 Python 中定义一个整型：``x = 100000``，``x`` 不是一个纯粹的整数，它是一个指向一个 C 语言结构体的指针。如果我们查看 Python 3.4 的源码就可以看到实现 Python 中的整型（实际上是长整型）的代码如下（其中的一些 C 语言宏已经被展开了）：

The standard Python implementation is written in C.
This means that every Python object is simply a cleverly-disguised C structure, which contains not only its value, but other information as well. For example, when we define an integer in Python, such as ``x = 10000``, ``x`` is not just a "raw" integer. It's actually a pointer to a compound C structure, which contains several values.
Looking through the Python 3.4 source code, we find that the integer (long) type definition effectively looks like this (once the C macros are expanded):

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

在 Python 3.4 中，一个简单的整型包含四个部分：

A single integer in Python 3.4 actually contains four pieces:

- ``ob_refcnt``, 引用计数器，在 Python 中用于内存管理
- ``ob_type``, 类型编码，表示了变量的实际类型
- ``ob_size``, 数据长度
- ``ob_digit``, 数值，这才是实际存储 Python 中整型的内容的地方。


- ``ob_refcnt``, a reference count that helps Python silently handle memory allocation and deallocation
- ``ob_type``, which encodes the type of the variable
- ``ob_size``, which specifies the size of the following data members
- ``ob_digit``, which contains the actual integer value that we expect the Python variable to represent.

与 C 语言相比，Python 存储一个整数占用了额外的空间，它们的对比如下所示：

This means that there is some overhead in storing an integer in Python as compared to an integer in a compiled language like C, as illustrated in the following figure:

![Integer Memory Layout](figures/cint_vs_pyint.png)

``PyObject_HEAD`` 包含了结构体中所示的引用计数器、类型编码以及其他上文所述的信息。


Here ``PyObject_HEAD`` is the part of the structure containing the reference count, type code, and other pieces mentioned before.

注意两者的区别：一个 C 语言中的整型表示了某个内存位置的内容是一个整数。而 Python 的整型再是一个指向内存中的一个结构体的指针，这个结构体包含了真正的数值。这些在结构体中包含的额外的信息赋予了 Python 的动态类型的特性。当然这些 Python 类型的附加信息都是有代价的，尤其是在操作大量数据的时候。

Notice the difference here: a C integer is essentially a label for a position in memory whose bytes encode an integer value.
A Python integer is a pointer to a position in memory containing all the Python object information, including the bytes that contain the integer value.
This extra information in the Python integer structure is what allows Python to be coded so freely and dynamically.
All this additional information in Python types comes at a cost, however, which becomes especially apparent in structures that combine many of these objects.

## Python List 不仅仅是数组
## A Python List Is More Than Just a List

让我们考虑使用一个包含了大量 Python 对象的数据结构的情况。list 是 Python 中的可变多元素容器。我们可以创建一个整数 list：

Let's consider now what happens when we use a Python data structure that holds many Python objects.
The standard mutable multi-element container in Python is the list.
We can create a list of integers as follows:

In [1]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

或者，一个字符串 list：

Or, similarly, a list of strings:

In [3]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

由于 Python 的动态类型特性，我们可以创建一个包含不同类型数据的 list：

Because of Python's dynamic typing, we can even create heterogeneous lists:

In [5]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

不过在这里灵活性却成为了一种负担：为了允许动态类型，list 中的每一个元素都需要包含它自己的类型信息、引用计数以及其他信息。在这里所有的变量的类型是一样的，每个元素所包含的类型信息是冗余的，如果可以用固定类型数组来保存则会高效的多。

动态类型 list 与固定类型 list (NumPy 中的 array)的区别如下：

But this flexibility comes at a cost: to allow these flexible types, each item in the list must contain its own type info, reference count, and other information–that is, each item is a complete Python object.
In the special case that all variables are of the same type, much of this information is redundant: it can be much more efficient to store data in a fixed-type array.
The difference between a dynamic-type list and a fixed-type (NumPy-style) array is illustrated in the following figure:

![Array Memory Layout](figures/array_vs_list.png)

本质上数组是一个指向一个连续数据区域的指针。Python 中 list 包含了一连串的指针，而其中的每个指针又指向了一个完整的 Python 对象，正如上文中提到的那样。当然，Python list 的优势在于灵活：每个元素都有自己的类型信息，list 中可以包含任意类型的元素。NumPy array 则缺乏这样的灵活性，但是在存储和使用方面都表现的更高效。

At the implementation level, the array essentially contains a single pointer to one contiguous block of data.
The Python list, on the other hand, contains a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer we saw earlier.
Again, the advantage of the list is flexibility: because each list element is a full structure containing both data and type information, the list can be filled with data of any desired type.
Fixed-type NumPy-style arrays lack this flexibility, but are much more efficient for storing and manipulating data.

## Fixed-Type Arrays in Python

## Python 中的固定类型数组

Python 提供了集中存储固定类型数据的方式。内建的 ``array`` 模块（在 Python 3.3 之后引入）可以被用于存储包含同一类型元素的数组：

Python offers several different options for storing data in efficient, fixed-type data buffers.
The built-in ``array`` module (available since Python 3.3) can be used to create dense arrays of a uniform type:

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

这里 ``'i'`` 表示数组内容的类型是整型。

Here ``'i'`` is a type code indicating the contents are integers.

而更常用的是 NumPy 中的 ``ndarray`` 类型。其不单单包含了类似于 Python ``array`` 的高效存储，还具备了大量操作数据的方法。在之后的章节中我们将具体的操作做解释；这里我们展示一些创建 NumPy 数组的方法。

首先我们引入 NumPy：

Much more useful, however, is the ``ndarray`` object of the NumPy package.
While Python's ``array`` object provides efficient storage of array-based data, NumPy adds to this efficient *operations* on that data.
We will explore these operations in later sections; here we'll demonstrate several ways of creating a NumPy array.

We'll start with the standard NumPy import, under the alias ``np``:

In [3]:
import numpy as np

## 利用 Python list 创建 NumPy array

我们可以用 ``np.array`` 从 Python list 创建 NumPy array：

## Creating Arrays from Python Lists

First, we can use ``np.array`` to create arrays from Python lists:

In [8]:
# integer array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

与 Python list 不同，NumPy array 的所有元素类型相同。如果类型不同 NumPy 会进行类型转换（这里，整型被转为浮点类型）：

Remember that unlike Python lists, NumPy is constrained to arrays that all contain the same type.
If types do not match, NumPy will upcast if possible (here, integers are up-cast to floating point):

In [9]:
np.array([3.14, 4, 2, 3])

array([ 3.14,  4.  ,  2.  ,  3.  ])

如果我们想显式声明数据的类型，可以采用 ``dtype``：

If we want to explicitly set the data type of the resulting array, we can use the ``dtype`` keyword:

In [10]:
np.array([1, 2, 3, 4], dtype='float32')

array([ 1.,  2.,  3.,  4.], dtype=float32)

NumPy 数组可以是多维数组；用 list 创建多维数组的方式如下：

Finally, unlike Python lists, NumPy arrays can explicitly be multi-dimensional; here's one way of initializing a multidimensional array using a list of lists:

In [11]:
# nested lists result in multi-dimensional arrays
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

内嵌的 list 成为了结果中二维数组的行。

The inner lists are treated as rows of the resulting two-dimensional array.

## 从头创建数组

## Creating Arrays from Scratch

对于大型的数组，利用 NumPy 模块中的方法创建数组更为高效。这里有几个例子：

Especially for larger arrays, it is more efficient to create arrays from scratch using routines built into NumPy.
Here are several examples:

In [4]:
# 创建一个长度为 10，数据类型为整型，数据全为 0 的数组
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [5]:
# 创建一个 3x5 的二维数组，数据类型为浮点型，数据全为 1
np.ones((3, 5), dtype=float)

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.,  1.]])

In [6]:
# 创建一个 3x5 的数组，数据全为 3.14
np.full((3, 5), 3.14)

array([[ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14],
       [ 3.14,  3.14,  3.14,  3.14,  3.14]])

In [9]:
# 创建一个由线性序列填充的数组
# 从 0 到 20，步长为 2
# 与 Python 内建的 range 方法类似

# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [10]:
# 创建一个从 0 到 1 均匀分割为 5 个元素的数组

# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([ 0.  ,  0.25,  0.5 ,  0.75,  1.  ])

In [17]:
# 采用均匀分布创建一个 3x3 的数组
# 默认均匀分布范围从 0 到 1

# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

array([[ 0.99844933,  0.52183819,  0.22421193],
       [ 0.08007488,  0.45429293,  0.20941444],
       [ 0.14360941,  0.96910973,  0.946117  ]])

In [18]:
# 采用高斯分布生成的值填充的 3x3 的数组
# 其中高斯分布的均值为 0 标准差为 1

# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[ 1.51772646,  0.39614948, -0.10634696],
       [ 0.25671348,  0.00732722,  0.37783601],
       [ 0.68446945,  0.15926039, -0.70744073]])

In [19]:
# 创建一个采用 [0, 10] 分为的随机数填充的 3x3 的数组

# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[2, 3, 4],
       [5, 7, 8],
       [0, 5, 0]])

In [20]:
# 创建一个 3x3 的单位矩阵

# Create a 3x3 identity matrix
np.eye(3)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [21]:
# 创建一个未初始化的数组，其中的值为所分配的内存的值

# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([ 1.,  1.,  1.])

## NumPy 标准数据类型

## NumPy Standard Data Types

NumPy 数组的元素类型相同，因此了解其数据类型以及每个类型的范围非常关键。NumPy 由 C 编写，其类型对于 C 与 Fortran 这样的用户来说非常熟悉。NumPy 的标准数据类型在下面的表格中展示。在构建一个数组时，可以用一个字符串指定其类型：

NumPy arrays contain values of a single type, so it is important to have detailed knowledge of those types and their limitations.
Because NumPy is built in C, the types will be familiar to users of C, Fortran, and other related languages.

The standard NumPy data types are listed in the following table.
Note that when constructing an array, they can be specified using a string:

```python
np.zeros(10, dtype='int16')
```

或者采用 NumPy 中的对象：

Or using the associated NumPy object:

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

除此之外还可以指定更高级的数据类型，例如数值是大端还是小端存储的；更多的信息可以在 [NumPy documentation](http://numpy.org/) 找到。NumPy 也支持复合型的数据类型，详见 [结构数据：NumPy 的 结构数组](02.09-Structured-Data-NumPy.ipynb)。

More advanced type specification is possible, such as specifying big or little endian numbers; for more information, refer to the [NumPy documentation](http://numpy.org/).
NumPy also supports compound data types, which will be covered in [Structured Data: NumPy's Structured Arrays](02.09-Structured-Data-NumPy.ipynb).

<!--NAVIGATION-->
< [Introduction to NumPy](02.00-Introduction-to-NumPy.ipynb) | [Contents](Index.ipynb) | [The Basics of NumPy Arrays](02.02-The-Basics-Of-NumPy-Arrays.ipynb) >